Import libraries.

In [28]:
import pandas as p
from gensim import matutils, models
import scipy.sparse
import pickle
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
#from nltk import word_tokenize, pos_tag
import nltk


data = p.read_pickle('data.pkl')
cv = CountVectorizer(stop_words = 'english')
data_cv = cv.fit_transform(data.speeches)
data_dtm = p.DataFrame(
    data_cv.toarray(),
    columns = cv.get_feature_names()
)
data_dtm.index = data.speaker
data_dtm


top_30 = {}
for col in dtm.columns:
    top = dtm[col].sort_values(ascending = False).head(30)
    top_30[col] = list(zip(top.index, top.values))
words = []
dtm = data_dtm.transpose()
for candidate in dtm.columns:
    top = [word for (word, count) in top_30[candidate]]
    for t in top:
        words.append(t)
words
add_stop_words = [word for word, count in Counter(words).most_common() if count > dtm.shape[1]/2]
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


# Let's read in our document-term matrix

In [2]:
# One of the required inputs is a term-document matrix
dtm = p.read_pickle('dtm_stop.pkl')
dtm_trans = dtm.transpose()
dtm_trans


speaker,Amy Klobuchar,Andrew Yang,Marianne Williamson,Tulsi Gabbard,Bernie Sanders,Pete Buttigieg,Kamala Harris,Michael Bloomberg,Cory Booker,Joe Biden,Beto O’Rourke,Elizabeth Warren,Julian Castro,Tom Steyer
aa,1,0,0,0,0,0,0,0,0,0,0,0,0,0
aapi,0,0,0,0,0,0,0,0,0,0,0,0,0,1
aaron,0,0,0,0,0,0,0,0,0,0,0,0,1,0
abandoned,0,0,0,0,0,1,1,0,0,0,0,0,1,1
abandoning,0,0,0,0,0,0,0,0,1,0,0,0,0,0
abated,0,0,0,0,0,0,0,0,0,1,0,0,0,0
abc,0,0,0,0,1,0,0,0,0,0,0,0,0,0
aberration,0,0,0,0,0,1,0,0,0,1,0,0,0,0
abhorrent,0,0,0,1,0,0,0,0,0,1,0,0,0,0
abide,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [3]:
# Gensim requires that you reformat the document term matrix:
sparse_counts = scipy.sparse.csr_matrix(dtm_trans)
corpus = matutils.Sparse2Corpus(sparse_counts)


In [4]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())


Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [37]:
# Let's create a function to pull out nouns from a string of text
def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)


In [38]:
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

# Read in the cleaned data, before the CountVectorizer step
data_clean = p.read_pickle('data.pkl')
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = p.DataFrame(data_clean.speeches.apply(nouns))
# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['people',
 'thats',
 'president',
 'going',
 'need',
 'trump',
 'think',
 'just',
 'make',
 'right',
 'country',
 'im',
 'want',
 'dont',
 'way',
 'know',
 'say',
 'like',
 'donald',
 'american']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words = stop_words)
data_cvn = cvn.fit_transform(data_nouns.speeches)
data_dtmn = p.DataFrame(data_cvn.toarray(), columns = cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn


,aa,aapi,aberration,abhorrent,ability,able,abolition,abominable,abortion,abortions,...,youre,youth,youtube,youve,zealand,zelensky,zero,zip,zone,zuckerberg
0,1,0,0,0,0,24,0,0,0,1,...,15,0,0,8,0,0,0,0,0,0
1,0,0,0,0,1,5,0,0,0,0,...,10,0,0,3,0,0,7,1,0,1
2,0,0,0,0,0,1,1,0,0,0,...,5,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,6,0,0,3,0,...,3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,2,13,0,0,1,0,...,27,1,1,5,0,0,0,1,0,0
5,0,0,1,0,3,18,0,0,0,0,...,36,0,0,5,1,0,2,0,2,0
6,0,0,0,0,15,12,0,0,0,0,...,5,0,0,1,0,0,0,0,0,0
7,0,0,0,0,1,3,0,1,0,0,...,7,0,0,0,0,0,0,0,0,0
8,0,0,0,0,2,1,0,0,0,0,...,11,0,0,3,0,0,1,0,0,0
9,0,0,1,1,6,88,0,0,1,0,...,19,0,0,6,0,0,1,1,3,0


In [39]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())


In [43]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics = 5, id2word = id2wordn, passes = 80)
ldan.print_topics()


[(0,
  '0.008*"sure" + 0.007*"texas" + 0.007*"care" + 0.006*"america" + 0.006*"able" + 0.005*"tax" + 0.004*"el" + 0.004*"paso" + 0.004*"law" + 0.004*"healthcare"'),
 (1,
  '0.020*"fact" + 0.009*"ive" + 0.009*"states" + 0.009*"number" + 0.009*"sure" + 0.009*"able" + 0.008*"time" + 0.008*"united" + 0.007*"world" + 0.007*"years"'),
 (2,
  '0.007*"plan" + 0.007*"states" + 0.007*"united" + 0.006*"time" + 0.006*"america" + 0.006*"issue" + 0.005*"sure" + 0.005*"justice" + 0.005*"years" + 0.005*"things"'),
 (3,
  '0.008*"america" + 0.008*"money" + 0.007*"time" + 0.005*"big" + 0.005*"americans" + 0.005*"things" + 0.005*"ive" + 0.005*"new" + 0.004*"states" + 0.004*"healthcare"'),
 (4,
  '0.010*"healthcare" + 0.008*"war" + 0.007*"years" + 0.006*"year" + 0.006*"america" + 0.006*"world" + 0.006*"change" + 0.006*"industry" + 0.005*"companies" + 0.005*"new"')]

In [ ]:
# Let's try 5 topics
ldan = models.LdaModel(corpus=corpusn, num_topics = 5, id2word = id2wordn, passes = 80)
ldan.print_topics()


Using nouns and adjectives, we were not very successful in identifying topics.
